In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import copy
from typing import Union
import random
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def clones(module, N):
    """Produce N identical layers."""
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

def attention(query, key, value):
    """注意力机制的实现, 输入分别是query, key, value, mask: 掩码张量,
       dropout是nn.Dropout层的实例化对象, 默认为None
    """
    # 在函数中, 首先取query的最后一维的大小, 一般情况下就等同于我们的词嵌入维度, 命名为d_k
    d_k = query.size(-1)
    # print("d_k:",d_k) #64
 
    # 按照注意力公式, 将query与key的转置相乘, 这里面key是将最后两个维度进行转置,
    # 再除以缩放系数根号下d_k, 这种计算方法也称为缩放点积注意力计算.
    # 得到注意力得分张量scores
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    # print("scores.shape",scores.shape) #torch.Size([2, 8, 4, 4])
    p_attn = F.softmax(scores, dim = -1)
    # 最后, 根据公式将p_attn与value张量相乘获得最终的query注意力表示, 同时返回注意力张量
    return torch.matmul(p_attn, value), p_attn
class MultiHeadedAttention(nn.Module):
    def __init__(self, head, embedding_dim):
        """在类的初始化时, 会传入三个参数，head代表头数，embedding_dim代表词嵌入的维度，
           dropout代表进行dropout操作时置0比率，默认是0.1."""
        super(MultiHeadedAttention, self).__init__()
 
        # 在函数中，首先使用了一个测试中常用的assert语句，判断h是否能被d_model整除，
        # 这是因为我们之后要给每个头分配等量的词特征.也就是embedding_dim/head个.
        assert embedding_dim % head == 0
        # 得到每个头获得的分割词向量维度d_k
        self.d_k = embedding_dim // head
        # 传入头数h
        self.head = head
        self.linears = clones(nn.Linear(embedding_dim, embedding_dim), 4)
 
        # self.attn为None，它代表最后得到的注意力张量，现在还没有结果所以为None.
        self.attn = None

    def forward(self, query, key, value):
        """前向逻辑函数, 它的输入参数有四个，前三个就是注意力机制需要的Q, K, V，
           最后一个是注意力机制中可能需要的mask掩码张量，默认是None. """
        batch_size = query.size(0)
        query, key, value = \
              [model(x).view(batch_size, -1, self.head, self.d_k).transpose(1, 2)
            for model, x in zip(self.linears, (query, key, value))]
        x, self.attn = attention(query, key, value)
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.head * self.d_k)

        return self.linears[-1](x)

In [15]:
# input = batch, seq, features
input = torch.randn(64, 10, 512)
muti = MultiHeadedAttention(4, 512)
output = muti(input, input, input)
output.shape

torch.Size([64, 10, 512])